<a href="https://colab.research.google.com/github/KamonohashiPerry/PyTorch/blob/master/PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorchのインストール
+ Colabの編集→ノートブックの設定でGPUモードにすることをお忘れなく。

In [0]:
# pip install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl

In [0]:
# pip install torchvision

In [0]:
# pip install tqdm

インストールできたかの確認

In [0]:
# import torch

In [0]:
# torch.tensor([1, 2, 3]).to("cuda:0")

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 145605 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.14-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.14-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
# drive mean root directory of  google drive
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive/"Colab Notebooks"/PyTorch

FashionMNIST  PyTorch_2.ipynb  PyTorch.ipynb  Untitled0.ipynb


In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

import warnings
warnings.filterwarnings('ignore')

# Chapter5
## Embedding

In [0]:
# 10000種類のトークンを20次元のベクトルで表現
emb = nn.Embedding(10000, 20, padding_idx=0) # トークンの数、embeddingの数

# Embedding層への入力はint64のTensor
inp = torch.tensor([1, 2, 5, 2, 10], dtype=torch.int64)

# 出力はfloat32のTensor
out = emb(inp)

In [0]:
out

tensor([[ 0.3071,  0.7800, -0.4079, -0.5493,  0.1104,  0.1936, -0.2991, -0.6101,
          0.3542,  0.2485,  0.6236,  2.1196,  1.0828,  0.6099,  0.4534, -0.3325,
         -0.5995,  0.5519, -1.2786, -0.1574],
        [-0.2907, -0.1377, -0.3279, -0.0847, -0.1782,  0.5321, -0.5360, -0.1514,
         -0.5574,  1.3574,  1.8059,  0.6890,  0.3260, -0.4898,  0.5074,  0.8951,
          0.0656,  0.0839, -0.4245, -0.7064],
        [ 1.3226,  0.2486, -2.8225,  0.0311,  0.5486, -1.2720, -0.8503, -0.4235,
          1.8996, -0.9953,  0.3081,  1.2451, -0.9237, -0.4128,  0.3147,  0.4796,
          0.5280,  0.4853,  0.7573, -1.3690],
        [-0.2907, -0.1377, -0.3279, -0.0847, -0.1782,  0.5321, -0.5360, -0.1514,
         -0.5574,  1.3574,  1.8059,  0.6890,  0.3260, -0.4898,  0.5074,  0.8951,
          0.0656,  0.0839, -0.4245, -0.7064],
        [ 1.3111,  0.7227,  0.9497, -1.3358,  1.1636, -0.8808,  0.4602,  1.1967,
         -1.3335,  1.1357,  0.4192,  0.0654,  2.0514, -1.8228,  0.4157, -0.6180,
      

In [0]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2019-12-03 11:37:47--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  19.5MB/s    in 7.1s    

2019-12-03 11:37:55 (11.3 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
!tar xf aclImdb_v1.tar.gz

In [0]:
ls

aclImdb/  aclImdb_v1.tar.gz  adc.json  drive/  sample_data/


In [0]:
# テキストデータを前処理するための関数の用意
# globモジュールはUnixシェルで使われているルールに従い指定されたパターンに一致するすべてのパス名を見つけ出すもの。
# pathlibモジュールはオブジェクト指向のファイルシステムパス。

import glob
import pathlib
import re

remove_marks_regex = re.compile("[,\.\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!])")

# 長い文字列をトークンIDのリストに変換する関数
def text2ids(text, vocab_dict):
  # !?以外の記号の削除
  text = remove_marks_regex.sub("", text)
  # !?と単語の間にスペースを挿入
  text = shift_marks_regex.sub(r" \l ", text)
  tokens = text.split()
  # ボキャブラリー含まれないトークンはID=0を割り当てる
  return [vocab_dict.get(token, 0) for token in tokens]

# IDのリストをint64のTensorに変換
def list2tensor(token_idxes, max_len=100, padding=True):
  # 各文章の分割語のトークンの数を制限
  if len(token_idxes) > max_len:
    token_idxes = token_idxes[:max_len]

  n_tokens = len(token_idxes)

  if padding:
    # 数が足りない場合は末尾を0で埋める
    token_idxes = token_idxes + [0]*(max_len - len(token_idxes))
  return torch.tensor(token_idxes, dtype=torch.int64), n_tokens

In [0]:
# ポジネガ分析用のDatasetクラスの作成
class IMDBDataset(Dataset):
  def __init__(self, dir_path, train=True,
               max_len=100, padding=True):
    self.max_len = max_len
    self.padding = padding

    path = pathlib.Path(dir_path)
    vocab_path = path.joinpath("imdb.vocab")

    # ボキャブラリファイルを読み込み、行ごとに分割
    self.vocab_array = vocab_path.open().read().strip().splitlines()

    # 単語をキーとし、値がIDのdictを作る
    self.vocab_dict = dict((w, i+1) for (i, w) in enumerate(self.vocab_array))

    if train:
      target_path = path.joinpath("train")

    else:
      target_path = path.joinpath("test")

    pos_files = sorted(glob.glob(str(target_path.joinpath("pos/*.txt"))))
    neg_files = sorted(glob.glob(str(target_path.joinpath("neg/*.txt"))))

    # posは1、negは0のlabelを付けて
    # (file_path, label)のtupleのリストを作成
    self.labeled_files = list(zip([0]*len(neg_files), neg_files)) + list(zip([1]*len(pos_files), pos_files))

  @property
  def vocab_size(self):
    return len(self.vocab_array)

  def __len__(self):
    return len(self.labeled_files)

  def __getitem__(self, idx):
    label, f = self.labeled_files[idx]

    # ファイルのテキストデータを読み取って小文字に変換
    data = open(f).read().lower()

    # テキストデータをIDのリストに変換
    data = text2ids(data, self.vocab_dict)

    # IDのリストをTensorに変換
    data, n_tokens = list2tensor(data, self.max_len, self.padding)

    return data, label, n_tokens


In [0]:
pwd

'/content'

In [0]:
# 訓練用とテスト用のDataLoaderの作成
train_data = IMDBDataset("/content/aclImdb/")
test_data = IMDBDataset("/content/aclImdb/", train=False)

train_loader = DataLoader(train_data, batch_size=32,
                          shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=32,
                          shuffle=False, num_workers=4)

In [0]:
# ネットワークの定義と訓練
# 入力XをEmbeddingでベクトルの時系列に変換して、これをRNNに入れて最後に出力が1次元の線形層につなげる

class SequenceTaggingNet(nn.Module):
  def __init__(self, num_embeddings,
               embedding_dim=50,
               hidden_size=50,
               num_layers=1,
               dropout=0.2):
    super().__init__()
    self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
    self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, dropout=dropout)
    self.linear = nn.Linear(hidden_size, 1)

  def forward(self, x, h0=None, l=None):
    # IDをEmbeddingで多次元のベクトルに変換する
    # xは(batch_size, step_size, embedding_size)
    # -> (batch_size, step_size, hidden_dim)
    x = self.emb(x)

    # 初期状態h0とともにRNNにxを渡す
    # xは(batch_size, step_size, embedding_dim)
    # -> (batch_size, step_size, hidden_dim)
    x, h = self.lstm(x, h0)

    # 最後のステップのみ取り出す
    # xは(batch_size, step_size, hidden_dim)
    # -> (batch_size, 1)
    if l is not None:
      # 入力のもともとの長さがある場合はそれを使用する
      x = x[list(range(len(x))), l-1, :]

    else:
      # なければ単純に最後を使用する
      x = x[:, -1, :] 

    # 取り出した最後のステップを線形層に入れる
    x = self.linear(x)

    # 余分な次元を削除する
    # (batch_size, 1) -> (batch_size, )
    x = x.squeeze()
    return x

In [0]:
# 訓練の作成

def eval_net(net, data_loader, device="cpu"):
  net.eval()
  ys = []
  ypreds = []

  for x, y, l in data_loader:
    x = x.to(device)
    y = y.to(device)
    l = l.to(device)

    with torch.no_grad():
      y_pred = net(x, l=l)
      y_pred = (y_pred > 0).long()
      ys.append(y)
      ypreds.append(y_pred)

    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [0]:
# 評価の作成

from statistics import mean

# num_embeddingsには0を含めてtrain_data.vocab+1を入れる
net = SequenceTaggingNet(train_data.vocab_size + 1, num_layers=2)
net.to("cuda:0")
opt = optim.Adam(net.parameters())
loss_f = nn.BCEWithLogitsLoss()

for epoch in range(10):
  losses = [] # 損失関数の初期化
  net.train() # ニューラルネットの初期化

  for x, y, l in tqdm.tqdm(train_loader):
    # データをGPUに送る
    x = x.to("cuda:0")
    y = y.to("cuda:0")
    l = l.to("cuda:0")

    y_pred = net(x, l=l) # 予測結果
    loss = loss_f(y_pred, y.float()) # 損失関数の計算
    net.zero_grad() # 前回のbackwardメソッドで計算された勾配の値を削除
    loss.backward() # 微分の計算
    opt.step() # 勾配を更新
    losses.append(loss.item()) # 損失関数のappend
  train_acc = eval_net(net, train_loader, "cuda:0") # 訓練における評価
  val_acc = eval_net(net, test_loader, "cuda:0") # バリデーションにおける評価
  print(epoch, mean(losses), train_acc, val_acc)

  0%|          | 0/782 [00:00<?, ?it/s]

0 0.6893604791835141 0.53125 0.0


  0%|          | 0/782 [00:00<?, ?it/s]

1 0.6694488896204688 0.78125 0.5625


  0%|          | 0/782 [00:00<?, ?it/s]

...

7 0.15167732976610437 0.96875 0.75


  0%|          | 0/782 [00:00<?, ?it/s]

8 0.11278490032024129 0.9375 0.71875


100%|██████████| 782/782 [00:16<00:00, 48.05it/s]


9 0.08585180637552915 1.0 0.78125


In [0]:
# RNNを使用しないモデルの作成

from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression

train_X, train_y = load_svmlight_file("/content/aclImdb/train/labeledBow.feat")

test_X, test_y = load_svmlight_file("/content/aclImdb/test/labeledBow.feat",
                                    n_features = train_X.shape[1])

model = LogisticRegression(C=0.1, max_iter=1000)
model.fit(train_X, train_y)
model.score(train_X, train_y), model.score(test_X, test_y)

(0.89888, 0.39596)

In [0]:
# 可変長の系列の扱い

class SequenceTaggingNet2(SequenceTaggingNet):

  def forward(self, x, h0=None, l=None):
    # IDをEmbeddingで多次元のベクトルに変換
    x = self.emb(x)

    # 長さ情報が与えられている場合はPackedSequenceを作る
    if l is not None:
      x = nn.utils.rnn.pack_padded_sequence(
          x, l, batch_first=True
      )

    # RNNに通す
    x, h = self.lstm(x, h0)

    # 最後のステップを取り出して線形層を入れる
    if l is not None:
      # 長さ情報がある場合は最後の層の内部状態のベクトルを直接利用できる
      # LSTMは通常の内部状態の他にブロックセルの状態もあるので、内部状態のみを使用する
      hidden_state, cell_state = h
      x = hidden_state[-1]

    else:
      x = x[:, -1, :]

    # 線形層に入れる
    x = self.linear(x).squeeze()
    return x

In [0]:
# 訓練部の作成

for epoch in range(10):
  losses = []
  net.train()
  for x, y, l in tqdm.tqdm(train_loader):
    # 長さの配列を長い順にソート
    l, sort_idx = torch.sort(l, descending=True)
    # 得られたインデクスを使用してx,yも並べ替え
    x = x[sort_idx]
    y = y[sort_idx]

    x = x.to("cuda:0")
    y = y.to("cuda:0")

    y_pred = net(x, l=l)
    loss = loss_f(y_pred, y.float())
    net.zero_grad()
    loss.backward()
    opt.step()
    losses.append(loss.item())

  train_acc = eval_net(net, train_loader, "cuda:0")
  val_acc = eval_net(net, test_loader, "cuda:0")
  print(epoch, mean(losses), train_acc, val_acc)

  0%|          | 0/782 [00:00<?, ?it/s]

0 0.06700480434343295 1.0 0.78125


  0%|          | 0/782 [00:00<?, ?it/s]

1 0.0515684007739653 1.0 0.75


  0%|          | 0/782 [00:00<?, ?it/s]

...

7 0.017573157786507792 1.0 0.78125


  0%|          | 0/782 [00:00<?, ?it/s]

8 0.016605320932117917 1.0 0.65625


100%|██████████| 782/782 [00:16<00:00, 46.21it/s]


9 0.019202219441865363 1.0 0.78125


## RNNによる文書生成
https://github.com/spro/practical-pytorch

In [0]:
# データ準備

# すべてのascii文字で辞書を作る
import string # 一般的な文字列操作
all_chars = string.printable # 印刷可能な ASCII 文字で構成される文字列
vocab_size = len(all_chars)
vocab_dict = dict((c, i) for (i, c) in enumerate(all_chars) )

# 文字列を数値のリストに変換する関数
def str2ints(s, vocab_dict):
  return [vocab_dict[c] for c in s] # ASCII文字のリストと照合して、そのインデックスを数字として返している

# 数値のリストを文字列に変換する関数
def inits2str(x, vocab_array):
  return "".join([vocab_array[i] for i in x]) # 数値のリストの要素ごとにjoinで結合して文字列を作る

In [37]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2019-12-04 13:22:15--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.07s   

2019-12-04 13:22:15 (14.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [0]:
# Datasetクラスの作成
# 巨大なテキストファイルを200文字などの文章に分割する

class ShakespearDataset(Dataset):
  def __init__(self, path, chunk_size=200):
    # ファイルを読み込み、数値のリストに変換する
    data = str2ints(open(path).read().strip(), vocab_dict) # strip()は文字列の先頭および末尾部分を除去したコピーを返す

    # Tensorに変換し、splitする
    data = torch.tensor(data, dtype=torch.int64).split(chunk_size)

    # 最後のchunkの長さをチェックして足り無い場合は捨てる
    if len(data[-1]) < chunk_size:
      data = data[:-1]

    self.data = data
    self.n_chunks = len(self.data)

  def __len__(self):
    return self.n_chunks

  def __getitem__(self, idx):
    return  self.data[idx]

In [0]:
# open("/content/input.txt").read().strip()

In [0]:
# DataLoaderの作成
ds = ShakespearDataset("/content/input.txt", chunk_size=200)
loader = DataLoader(ds, batch_size=32, shuffle=True, num_workers=4)

In [0]:
# モデルの構築

class SequenceGenerationNet(nn.Module):
  def __init__(self, num_embeddings,
               embedding_dim=50,
               hidden_size=50,
               num_layers=1,
               dropout=0.2):
    super().__init__()
    # embedding層
    self.emb = nn.Embedding(num_embeddings, embedding_dim)
    # LSTM層
    self.lstm = nn.LSTM(embedding_dim,
                        hidden_size,
                        num_layers,
                        batch_first=True,
                        dropout=dropout)
    # Linearのoutputのサイズは最初のEmbeddingのinputサイズと同じnum_embeddings
    self.linear = nn.Linear(hidden_size, num_embeddings)

  def forward(self, x, h0=None):
    x = self.emb(x) # embedding
    x, h = self.lstm(x, h0) # LSTM
    x = self.linear(x) # LSTMの出力に対して線形変換をする
    return x, h

In [0]:
# 文章を生成する関数

def generate_seq(net, start_phrase="The King said ", 
                 length=200, temperature=0.8, device="cpu"):
  # モデルを評価モードにする
  net.eval()

  # 出力の数値を格納するリスト
  result = []

  # 開始文字列をTensorに変換
  start_tensor = torch.tensor(
      str2ints(start_phrase, vocab_dict),
      dtype=torch.int64
  ).to(device)

  # 戦闘にbatch次元を付ける
  x0 = start_tensor.unsqueeze(0)

  # RNNに通して出力と新しい内部状態を得る
  o, h = net(x0)

  # 出力を（正規化されていない）確率に変換
  out_dist = o[:, -1].view(-1).exp()

  # 確率から実際の文字のインデクスをサンプリング
  top_i = torch.multinomial(out_dist, 1)[0] # 多項分布からサンプリング

  # 結果を保存
  result.append(top_i)

  # 生成された結果を次々にRNNに入力していく
  for i in range(length):
    inp = torch.tensor([[top_i]], dtype=torch.int64)
    inp = inp.to(device)
    o, h = net(inp, h)
    out_dist = o.view(-1).exp()
    top_i = torch.multinomial(out_dist, 1)[0]
    result.append(top_i)

  # 開始文字列と生成された文字れるをまとめて返す
  return start_phrase + inits2str(result, all_chars)

In [0]:
# モデルの訓練

from statistics import mean

net = SequenceGenerationNet(vocab_size, 20, 50,
                            num_layers=2, dropout=0.1)
net.to("cuda:0")

opt = optim.Adam(net.parameters())

# 多クラスの識別で問題なのでSoftmaxCrossEntropyLossが損失関数となる
loss_f = nn.CrossEntropyLoss()

for epoch in range(50):
  net.train()
  losses = []
  for data in tqdm.tqdm(loader):
    # xは始めから最後の手前の文字まで
    x = data[:, :-1]
    # yは2文字目から最後の文字まで
    y = data[:, 1:]

    x = x.to("cuda:0")
    y = y.to("cuda:0")

    y_pred, _ = net(x)

    # batchとstepの軸を統合してからCrossEntropyLossにわたす
    loss = loss_f(y_pred.view(-1, vocab_size), y.view(-1)) # 損失関数の計算
    net.zero_grad() # 前回の勾配をリセット
    loss.backward() # 自動微分
    opt.step() # 更新
    losses.append(loss.item()) # 損失関数へのappend

  # 現在の損失関数と生成される文章の例を表示
  print(epoch, mean(losses))
  with torch.no_grad(): # set all the requires_grad flag to false
    print(generate_seq(net, device="cuda:0"))

100%|██████████| 175/175 [00:14<00:00, 12.75it/s]49 1.6556574508122035
The King said their make,
My juck, I'll be make.

GRINE RICES:
That quike a's I, seem, I speak thou awain.

EXHO:
Go as you world or a-tay purfo?
Jeting him speaks oum't of thit forth purther?
Where of liegion this 